# 1. Дата выполнения лабораторной работы

> 17.11.2024


# 2. Название лабораторной работы

> "Введение в машинное обучение. Основы Pandas"


# 3. Цель лабораторной работы

> Изучить методы анализа и обработки табличных данных с помощью библиотек на языке программирования python для дальнейшего моделирования.


# 4. Задачи лабораторной работы

> 1. Освоить способы получения информации из табличных данных. Ознакомиться со способами хранения информации.
> 2. Освоить способы визуализации табличных данных.
> 3. Освоить способы обработки данных (обработка возможных пропусков, приведение к нужным типам данных).



# 5. Материалы, использовавшиеся для выполнения лабораторной работы (программы, исходные файлы, библиотеки). 

* Для программ, языков программирования, библиотек указываются версии ПО.

В данной работе использовались:
* [база данных ChEMBL](https://www.ebi.ac.uk/chembl/explore/compound/CHEMBL25)
* [Туториал по библиотеке сhembl на языке программирования python](https://github.com/chembl/notebooks/blob/main/ChEMBL_webresource_client_examples.ipynb)
* [THE CHEMBL-OG](https://chembl.blogspot.com/2020/03/chembl-26-released.html)

> * chembl_webresource_client==0.10.9 
> * pandas==2.2.2
> * numpy==1.26.4
> * matplotlib==3.9.0
> * seaborn==0.13.2
> * tqdm==4.66.5


#  6. Поэтапное изложение процедуры — связным текстом. 
* Для лабораторных работ, включающих построение QSAR/QSPR моделей, изложение процедур должно включать следующие пункты:
    * описание набора данных
    * описание подготовки молекул и способов считать дескрипторы
    * описание методов машинного обучения (как строились модели, какие методы использовались, иная важная информация касательно моделирования)
    * оценка качества моделей. Какие метрики качества использовались. Что они значат. Как проводилось разбиение на обучающую и тестовую выборки.
    * домен применимости (если использовался)

Мне были предоставлены данные в виде таблицы (он же датафрейм) с химической базы данных  СhEMBL в формате .csv. 

В начале работы был произведен первичный взгляд на данные: были построены гистограммы, рассчитан процент пропусков. Визуализация пропусков через проценты показала, что некоторые признаки имеют одинаковый процент пропусков. Мною было выдвинуто следующее предположение: эти признаки одинаково отсуствуют во всей строке у молекулы, то есть их не получится восстановить друг через друга.
Далее выполнялась работа по обработке данных: названия колонок приводились к стилю языка python, были преобразованы типы данных до более меньших возможных по памяти типов. Названия колонок были преобразованы в соотвествии с базой данных ChEMBL. Неинформативные признаки (они же колонки или дескрипторы) были удалены.

Была произведена попытка по восполнению данных с помощью библиотеки на языке python (chembl), но она оказалась провальной. Из-за большого количества https-запросов могла возникать ошибка 404.

        `Ошибка при обработке CHEMBL3545296: HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/CHEMBL3545296 (Caused by ResponseError('too many 404 error responses'))`

Мною было выявлено, что начальный датафрейм содержал в себе информацию об органических и неорганических молекулах. Были выгружены данные со всеми их минимальным, максимальным и уникальными значениями. По этим значениям можно было понять какой тип данных подходит определенным дескрипторам. Было принято решение о полном удалении молекулы (строки) из таблицы, если отсутсвовало больше 50% информации о ней. После этого колонка `inorganic_flag` стала принимать только одно значение - 0. То есть пропала информация о неорганических молекулах. Колонка `type` содержала в себе только информацию о малых молекулах. Колонка `structure_type` содержала в себе либо `MOL`, либо `NONE`. Эти колонки были удалены потому, что по сути содержали в себе лишь одно значение. Для машинного обучения они являются бесполезными, так как нет других значений, а обучаться на одном единственном значении для всех молекул - бессмысленно и бесполезно. Были также удалены колонки 'synonyms', 'type', 'structure_type', 'canonical_smiles',	'standard_inchi_key', 'name', 'full_molformula', так как для машинного обучения эти данные не несут в себе смысла. Модель обучется на числах: вещественных, категоральных, но не на строковых. Несмотря на то, что существуют модели на основе обработки текстовой информации. В данной работе предполагается дальнейшее использование модели, которая обучается на числах. Очень важно обрабатывать данные в соотвествии с предполагаемой целевой метрикой и методом машинного обучения. Была произведена попытка восстановить молекулярный вес по молекулярной формуле молекулы, но в этих значениях оказывались одновременно пропуски (процент пропуска в этих признаках был одинаковым - 6%). Признаки `ro3_pass` и `molecular_species` были закодированы: преобразованы в числовой тип данных: int8. Существует еще тип данных `category`. Но, если данные являются числами и укладываются в диапазон int8, то int8 всегда занимает меньше памяти, чем category, так как не требует хранения таблицы уникальных значений.

Далее был проведен анализ данных: былы построены тепловая картка, графики корреляции, гистограммы, ящики с усами. Корреляция вычислялась методом Спирмена, так как данные были скошены. Были удалены колонки 'targets', 'bioactivities', так как после проведения дополнительного анализа осталось непонятным что за ними скрывается в числовом формате. По базе данных предполагается, что под `targets` скрывается название мишени, аналогично с `bioactivities`. На некоторых графиках видны отклонения. Это либо из-за химической структуры, либо из-за погрешностей измерений, представленных в базе данных ChEMBL.





# 7. Результаты работы
 * (+ таблицы, графики, иллюстрации, если требуются)

> * В конце датафрейм содержал в себе исключиетльно информацию и малых органических молекулах. Вся возможная информация была приведена к числовому типу (закодирована), остальная была удалена. Итоговый датафрейм был сохранен в формате pickle для дальнейшего моделирования. Из 1000 молекул осталось только 320. Процент удаленных данных о молекулах составил: 68.00%. После обработки объем данных снизился с 0,84 Мб до 0.01 Мб.

# 8. Выводы

> * В ходе работы я провела обработку данных в помощью нескольких библиотек на языке программирования python. Я столкнулась с тем, что можно некорректно подготовить данные, если не знать их дальнейшую судьбу: какое именно моделирование будет использоваться, какая целевая метрика. Также важно знать точно когда  и откуда были собраны эти данные (насколько данные являются актуальными). Мною была выполнена предобработка данных на моих личных предположениях и упрощениях.